In [1]:
from tree_sitter import Parser, Language
import tree_sitter_java as tsj
from neo4j import GraphDatabase

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

In [2]:
language = Language(tsj.language())
parser = Parser(language=language)
parser

In [57]:
with open("../data/java/employee-crud/src/main/java/com/java_ast_knowledge_graph_poc/employee_crud/controller/EmployeeController.java", "r", encoding="utf-8") as file:
    source_code = file.read()

source_code

'package com.java_ast_knowledge_graph_poc.employee_crud.controller;\n\nimport java.util.List;\n\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.http.HttpStatus;\nimport org.springframework.http.ResponseEntity;\nimport org.springframework.web.bind.annotation.DeleteMapping;\nimport org.springframework.web.bind.annotation.GetMapping;\nimport org.springframework.web.bind.annotation.PathVariable;\nimport org.springframework.web.bind.annotation.PostMapping;\nimport org.springframework.web.bind.annotation.PutMapping;\nimport org.springframework.web.bind.annotation.RequestBody;\nimport org.springframework.web.bind.annotation.RequestMapping;\nimport org.springframework.web.bind.annotation.RestController;\n\nimport com.java_ast_knowledge_graph_poc.employee_crud.dto.EmployeeDTO;\nimport com.java_ast_knowledge_graph_poc.employee_crud.service.EmployeeService;\n\n@RestController\n@RequestMapping("/api/employees")\npublic class EmployeeController {\n\n    p

In [58]:
tree = parser.parse(bytes(source_code, "utf8"))
root_node = tree.root_node

In [59]:
graphDBConn = Neo4jConnection(uri="neo4j://localhost:7687", user="neo4j", password="password")

In [ ]:
graphDBConn.driver.execute_query("MATCH p=()-[]->() RETURN p")

EagerResult(records=[<Record p=<Path start=<Node element_id='4:006ce47f-db97-446a-8d48-00e2d825b218:1' labels=frozenset({'Method'}) properties={'name': 'main', 'content': 'System.out.println("Hello World");\n'}> end=<Node element_id='4:006ce47f-db97-446a-8d48-00e2d825b218:0' labels=frozenset({'Class'}) properties={'name': 'HellowWorld.java'}> size=1>>], summary=<neo4j._work.summary.ResultSummary object at 0x11491e0d0>, keys=['p'])

In [60]:
children_nodes = root_node.children
class_declration_node = children_nodes[-1]
package_declaration_node = children_nodes[0]

create package node

In [62]:
print(f"name: {package_declaration_node.children[1].text.decode('utf-8')}")
print(f"content: {package_declaration_node.text.decode('utf-8')}")

package_node = graphDBConn.driver.execute_query(
    "CREATE (p:Package {name: $name, content: $content}) RETURN p",
    name=package_declaration_node.children[1].text.decode('utf-8'),
    content=package_declaration_node.text.decode('utf-8')
)

package_node

name: com.java_ast_knowledge_graph_poc.employee_crud.controller
content: package com.java_ast_knowledge_graph_poc.employee_crud.controller;


EagerResult(records=[<Record p=<Node element_id='4:006ce47f-db97-446a-8d48-00e2d825b218:2' labels=frozenset({'Package'}) properties={'name': 'com.java_ast_knowledge_graph_poc.employee_crud.controller', 'content': 'package com.java_ast_knowledge_graph_poc.employee_crud.controller;'}>>], summary=<neo4j._work.summary.ResultSummary object at 0x1140395b0>, keys=['p'])

In [ ]:
package_node.records[0].get("p")

<Node element_id='4:006ce47f-db97-446a-8d48-00e2d825b218:2' labels=frozenset({'Package'}) properties={'name': 'com.java_ast_knowledge_graph_poc.employee_crud.controller', 'content': 'package com.java_ast_knowledge_graph_poc.employee_crud.controller;'}>

create class node

In [106]:
rest_path = class_declration_node.children[0].children[1].children[-1].children[1].children[1].text.decode('utf-8')
class_name =class_declration_node.children[-2].text.decode('utf-8')
content = " ".join(list(map(lambda x: x.text.decode('utf-8'), class_declration_node.children[:-1])))
# Get the package node ID from the previous query
package_id = package_node.records[0].get("p").element_id

# Create class and relationship in one query
class_node = graphDBConn.driver.execute_query(
    """
    MATCH (pkg:Package) WHERE elementId(pkg) = $package_id
    CREATE (c:Class {name: $name, rest_path: $rest_path, content: $content})
    CREATE (c)-[:BELONGS_TO]->(pkg)
    RETURN c
    """,
    package_id=package_id,
    name=class_name,
    rest_path=rest_path,
    content=content
)

class_node.records[0].get("c")

<Node element_id='4:006ce47f-db97-446a-8d48-00e2d825b218:0' labels=frozenset({'Class'}) properties={'name': 'EmployeeController', 'content': '@RestController\n@RequestMapping("/api/employees")\npublic class EmployeeController', 'rest_path': '/api/employees'}>